# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley, 2016

# 1) Monty: The Python Siri

Let's make a Siri-like program with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [1]:
import wave
import pyaudio
import houndify
from keys import client_id, client_key, user, pwd # contains Houndify API info and gmail username/password
import json
import smtplib
from email.mime.text import MIMEText
import urllib.request
from test_responses import *

In [2]:
def record_audio(fname, rec_sec=10):
    """Record 10 seconds of audio data and writes it to file"""
    chunk = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = rec_sec
    WAVE_OUTPUT_FILENAME = fname
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("* done recording")
    stream.close()
    p.terminate()
    data = b"".join(all)
    wf = wave.open(WAVE_OUTPUT_FILENAME, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [3]:
def do_math(response):
    """
    Returns the formula and result that Houndify understood and calculated
    `response` is the JSON response from Houndify
    """
    formula = response['AllResults'][0]['NativeData']['Formula']
    result = response['AllResults'][0]['NativeData']['Result']
    return formula+'='+result

In [54]:
def send_email(response):
    """
    Sends an email with the information understood by Houndify.
    `response` is the JSON response from Houndify.
    `user` and `pwd` are not included in the repo for obvious reasons; they can be replaced
    with your information.
    This function will pause to make sure the user wants to send the email.
    """
    try:
        email_info = response['AllResults'][0]['NativeData']['Email']
    except KeyError:
        print("Houndify didn't understand that...try again.")
        return
    subject = email_info['Subject']
    body = email_info['Body']
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = user
    msg['To'] = user
    server = smtplib.SMTP('smtp.gmail.com', 587, None, 30)
    server.starttls()
    server.login(user, pwd)
    server.sendmail(user, user, msg.as_string())
    print(msg.as_string())
    server.quit()
    return msg.as_string()

In [55]:
def get_joke():
    """
    Get a joke from http://tambal.azurewebsites.net/joke/random
    """
    contents = urllib.request.urlopen('http://tambal.azurewebsites.net/joke/random').read()
    joke = json.loads(contents.decode('utf-8'))
    return joke['joke']

In [56]:
def respond(response):
    """
    Using the Houndify structured response, determines which actions to take.
    """
    if len(response['DomainUsage']) != 0:
        if response['DomainUsage'][0]['Domain'] == 'Email':
            send_email(response)
        if response['DomainUsage'][0]['Domain'] == 'Arithmetic':
            print(do_math(response))
    else:
        transcript = response['Disambiguation']['ChoiceData'][0]['Transcription']
        if 'joke' in transcript:
            print(get_joke())
        else:
            "Sorry, I didn't get that..."
    return response

In [57]:
def test_all_response_types():
    for response in [joke_response, math_response, email_response2, email_response]:
        r = respond(response)
        print('\n'+'-'*70+'\n')
    return r

In [58]:
r = test_all_response_types()

What's funnier than a monkey dancing with an elephant? Two monkeys dancing with an elephant.

----------------------------------------------------------------------

9147*474=4.33568e+06

----------------------------------------------------------------------

Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit
Subject: Hello
From: samdixon526@gmail.com
To: samdixon526@gmail.com

Goodbye

----------------------------------------------------------------------

Houndify didn't understand that...try again.

----------------------------------------------------------------------



In [59]:
clientId = client_id
clientKey = client_key
userId = "test_user"
requestInfo = {'ClientID': clientId,'UserID': userId}
client = houndify.StreamingHoundClient(clientId, clientKey, userId, requestInfo, sampleRate=16000)

class Listener(houndify.HoundListener):
    def onPartialTranscript(self, transcript):
        pass
    def onFinalResponse(self, response):
        respond(response)
    def onError(self, err):
        print('Error:', err)

def ask_houndify(fname):
    chunk = 1024
    l = Listener()
    client.start(l)
    f = wave.open(fname, 'rb')
    rate = f.getframerate()
    samples = f.readframes(chunk)
    finished = False
    while not finished:
        client.fill(samples)
        samples = f.readframes(chunk)
        if len(samples)==0:
            break
    client.finish()

In [60]:
def test_ask_houndify():
    fnames = ['email.wav', 'math.wav', 'joke.wav']
    for f in [os.path.join('./test_queries', f) for f in fnames]:
        ask_houndify(f)

In [61]:
test_ask_houndify()

Houndify didn't understand that...try again.
9147*474=4.33568e+06
My friend's bakery burned down last night. Now his business is toast.


In [62]:
def do_all():
    wait = input('Press enter to start recording')
    if wait == '':
        record_audio('tmp.wav')
    print('Sending to Houndify')
    ask_houndify('tmp.wav')
    print('Done!')

In [64]:
do_all()

Press enter to start recording
* done recording
Sending to Houndify
What do I look like? A JOKE MACHINE!?
Done!


# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/fall2016/ay250/resources -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.

In [ ]:
import aifc
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from detect_peaks import detect_peaks
%matplotlib inline

In [ ]:
def read_aiff(path, chunk_size=1024, dtype=np.int32, byteswap=True):
    """
    Read in AIFF files one chunk at a time. The data is 32 bit, and byteswapping is needed
    to properly convert hexcode to strings
    """
    CHUNK = chunk_size
    f = aifc.open(path, 'rb')
    frame_rate = f.getframerate()
    nchannels = f.getnchannels() # Sound files are in stereo
    result = []
    data = f.readframes(CHUNK)
    while len(data) > 0:
        result.append(np.fromstring(data, dtype=np.int32).byteswap())
        data = f.readframes(CHUNK)
    result = np.concatenate(result)
    return result, frame_rate*nchannels

In [ ]:
def find_peaks(data, frequencies, plot=True, remove_harmonics=True):
    """
    Detect the peak frequencies in data.
    Plotting shows the Fourier transform with the detected peaks highlighted.
    This code makes use of the detect_peaks function from Marcos Duarte
    (https://github.com/demotu/BMC)
    """
    N = len(data)*2
    T = 1.0 / frequencies
    x = np.linspace(0.0, N*T, N)
    y = data
    yf = fft(data)
    xf = np.linspace(0.0, 1.0/(2.0*T), N/2)
    yf = 2.0/N*np.abs(yf[(60<xf) & (xf<10000)])
    xf = xf[(60<xf) & (xf<10000)]
    ind = detect_peaks(yf, mph=3e6, mpd=500) # mph: minimum peak height, mpd: minimum peak distance
    if plot:
        plt.plot(xf, yf, 'k-', alpha=0.7)
        plt.plot(xf[ind], yf[ind], 'ro')
        plt.xlim(60, 4000)
        plt.xscale('log')
        plt.yscale('log')
    if remove_harmonics:
        # Harmonic removal isn't quite perfect...
        peak_freq = xf[ind]
        no_h = [peak_freq[0]]
        for x in peak_freq[1:]:
            # Heads up: this loop alters the list it loops over
            peak_freq = list(filter(lambda x: x/peak_freq[0]%1>0.1 and x/peak_freq[0]%1<0.9, peak_freq))
            if len(peak_freq)!=0:
                no_h.append(peak_freq[0])
        plt.plot(no_h, yf[np.in1d(xf, no_h)], 'go')
        return no_h, yf[np.in1d(xf, no_h)]
    else: return xf[ind], yf[ind]

In [ ]:
def create_note_dict(a=440, reverse=False):
    """
    Calculate the note name for various frequencies and return as a dictionary.
    """
    midi = [(a / 8) * (2 ** ((x - 9) / 12)) for x in range(127)]
    notes = ['C',
             'C#/Db',
             'D',
             'D#/Eb',
             'E',
             'F',
             'F#/Gb',
             'G',
             'G#/Ab',
             'A',
             'A#/Bb',
             'B']
    note_dict = {}
    for i, m in enumerate(midi):
        octave = i//12+1
        note = notes[i%12]
        name = note+str(octave)
        if reverse:
            note_dict[name] = m
        else:
            note_dict[m] = name
    return note_dict

In [ ]:
def match_notes(peaks):
    """
    Match peak frequencies to the closest note name.
    """
    note_dict = create_note_dict()
    note_names = []
    for p in peaks:
        note_names.append(note_dict[min(note_dict.keys(), key=lambda k: abs(k-p))])
    return peaks, note_names

In [ ]:
test_files = ['A4_PopOrgan.aif', 'C4+A4_PopOrgan.aif', 'F3_PopOrgan.aif', 'F4_CathedralOrgan.aif']
for path in [os.path.join('./sound_files/', f) for f in test_files]:
    print(os.path.basename(path)[:-4])
    data, frequencies = read_aiff(path)
    peak_freq, peak_height = find_peaks(data, frequencies)
    _, notes = match_notes(peak_freq)
    plt.show()
    print(notes)

In [ ]:
for i in range(1, 13):
    fname = os.path.join('./sound_files/', str(i)+'.aif')
    data, frequencies = read_aiff(fname)
    peak_freq, peak_height = find_peaks(data, frequencies, remove_harmonics=True)
    _, notes = match_notes(peak_freq)
    plt.show()
    print(fname, notes)